In [54]:
seed = 0

import os
import sys

os.chdir('..')
utils_path = os.path.abspath("./utilities/")
sys.path.append(utils_path)

from load_data import load_fullECAI
from evaluation import *
from evaluation import _my_scorer

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, Normalizer
from sklearn.metrics import euclidean_distances
import joblib
from joblib import Parallel, delayed
import shap

from igraph import Graph
import igraph as ig

In [10]:
def mean_degree(g):
    degree = g.degree_distribution()
    
    return degree.mean

def frac_nodes_large_component(g):
    return


def mean_dist_node_pairs(g):
    return


def exp_power_law(g):
    return


def cluster_coeff(g):
    return g.transitivity_undirected()


def random_cluster_coeff(g):
    n = g.vcount()
    degrees = g.degree(g.vs, loops=False)
    unique_degrees, degree_freqs = np.unique(degrees, return_counts=True)
    prop_degrees = degree_freqs/n
    moment_1 = np.dot(unique_degrees, prop_degrees)
    moment_2 = np.dot(unique_degrees**2, prop_degrees)
    
    return (1/n) * ((moment_2 - moment_1)**2) / (moment_1**3)



def cluster_coeff_alt(g):
    return


def modularity(g):
    # Checar si este es con distancias o fuerzas
    mod = g.modularity(y_test.to_list(), weights=g.es['weight'])
    return mod

def summary(result, model_name):
    summary_stats = {}
    summary_stats['modelo'] = model_name
    summary_stats['corte'] = result[0]
    summary_stats['global_efficiency'] = result[1]
    summary_stats['aristas'] = result[2]
    summary_stats['mean_degree'] = mean_degree(result[3])
    summary_stats['S'] = frac_nodes_large_component(result[3])
    summary_stats['mean_dist_node_pairs'] = mean_dist_node_pairs(result[3])
    summary_stats['exp_power_law'] = exp_power_law(result[3])
    summary_stats['cluster_coeff'] = cluster_coeff(result[3])
    summary_stats['cluster_coeff_alt'] = cluster_coeff_alt(result[3])
    summary_stats['modularity'] = modularity(result[3])
    
    return summary_stats

In [69]:
dataset = load_fullECAI()
# Prep data
X = dataset.drop('status', axis=1)
y = dataset.loc[:, 'status']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=seed, stratify=y)

In [55]:
models_paths = ['./results/XGBoost/GridSearchCV_22-03-22_03-27-03/models/best/best_xgb.pkl', 
                './results/Basic/models/RandomForestClassifier.pkl', 
                './results/Basic/models/LogisticRegression.pkl',
                './results/Basic/models/LinearDiscriminantAnalysis.pkl',
                './results/Basic/models/XGBClassifier.pkl']

In [56]:
model_folder = '/'.join(model.split('/')[:-1])
model_name = model.split('/')[-1].split('.')[0]
results = joblib.load('{}/{}_reduced_graphs_efficiency.pkl'.format(model_folder, model_name))

In [65]:
g = results[0][3]

In [77]:
n = y_test.shape[0]

In [33]:
models_stats = []

for model in models_paths:
    model_folder = '/'.join(model.split('/')[:-1])
    model_name = model.split('/')[-1].split('.')[0]
    results = joblib.load('{}/{}_reduced_graphs_efficiency.pkl'.format(model_folder, model_name))
    
    ## Stats per cutoff
    stats_tmp = Parallel(n_jobs=4)(delayed(summary)(r, model_name) for r in results)
    
    ## Save in list
    models_stats = models_stats + stats_tmp

In [19]:
#pd.DataFrame(columns=['Modelo', 'Corte', 'Efficiencia', 'Optimo', 'Aristas', 'Mean Degree', 'S', 'Distancia promedio, nodos pares', 'Exponente Power law', 'Clustering Coefficient', 'Clustering Coefficient alt', 'Modularidad'])

,Modelo,Corte,Efficiencia,Optimo,Aristas,Mean Degree,S,"Distancia promedio, nodos pares",Exponente Power law,Clustering Coefficient,Clustering Coefficient alt,Modularidad
